### Indexed JSON Convert to GLB

We include a small snippet here to convert an Indexed JSON file (from our custom cuda_voxelizer) to a GLB. 

This is especially helpful when debugging voxel locations or colors via GLB output.

In [7]:
import json
import numpy as np
import trimesh

def load_indexed_json(json_file):
    """Load the JSON file and extract voxel data."""
    with open(json_file, 'r') as f:
        data = json.load(f)

    blocks = data['blocks']
    xyzi = data['xyzi']

    voxel_positions = []
    voxel_colors = []

    # Extract voxel positions and colors
    for voxel in xyzi:
        x, y, z, color_index = voxel
        voxel_positions.append([x, y, z])

        # Retrieve the color corresponding to the color index
        rgba = blocks[str(color_index)]
        r, g, b = rgba[:3]  # Ignore alpha for now
        # Assign two identical colors per face (12 colors per voxel for 6 faces)
        voxel_colors.extend([[r / 255.0, g / 255.0, b / 255.0, 1.0]] * 12)

    return np.array(voxel_positions), np.array(voxel_colors)

def create_voxel_mesh(positions, colors):
    """Create a voxel mesh using trimesh."""
    voxel_size = 1.0  # Define each voxel as a unit cube

    # Create a voxel grid by adding cubes at specified positions
    voxels = []
    for pos in positions:
        voxel = trimesh.creation.box(extents=[voxel_size] * 3)
        voxel.apply_translation(pos)
        voxels.append(voxel)

    if not voxels:
        raise ValueError("No voxels to create a mesh.")

    # Combine all the voxel cubes into a single mesh
    combined_mesh = trimesh.util.concatenate(voxels)

    # Assign the colors to each face of the mesh
    if len(colors) != len(combined_mesh.faces):
        raise ValueError(f"Number of colors ({len(colors)}) does not match number of faces ({len(combined_mesh.faces)}).")
    
    combined_mesh.visual.face_colors = colors

    return combined_mesh

def save_as_glb(mesh, output_file):
    """Save the mesh as a GLB file."""
    mesh.export(output_file, file_type='glb')
    print(f"Saved GLB to {output_file}")

def main():
    input_json = 'voxel_data.json' # REPLACE !!! 
    output_glb = 'voxel_visualization.glb'

    # Load JSON data and create a mesh
    positions, colors = load_indexed_json(input_json)
    
    print(f"Loaded {len(positions)} voxel positions.")
    print(f"Loaded {len(colors)} voxel colors.")

    voxel_mesh = create_voxel_mesh(positions, colors)

    # Save the mesh as a GLB file for visualization
    save_as_glb(voxel_mesh, output_glb)

if __name__ == '__main__':
    main()


Loaded 36155 voxel positions.
Loaded 433860 voxel colors.
Saved GLB to voxel_visualization.glb
